[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichardJPovinelli/Neural_Networks_Course/blob/main/Attention_Backprop_Demo.ipynb)
# Backpropagation Through Scaled Dot-Product Attention (Demo)

This notebook demonstrates **backpropagation through a tiny self-attention block**.

We will:

1. Define a toy self-attention module (single head) with small, fixed matrices.
2. Run a forward pass and define a simple scalar loss.
3. Use PyTorch autograd to compute gradients.
4. Manually compute the same gradients using the analytical formulas for attention.
5. Compare **autograd vs. manual vs. finite-difference** to verify everything matches.


In [ ]:
import torch
import math

PRECISION = 2

torch.set_printoptions(precision=PRECISION, sci_mode=True)
# choose best available device automatically
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"  # Apple Silicon GPU
else:
    device = "cpu"
print("Using PyTorch version:", torch.__version__)


Using PyTorch version: 2.9.0+cpu



## 1. Tiny Self-Attention Setup

We use:
- Sequence length $n = 3$
- Model dimension $d_{model} = d_k = d_v = 4$
- Single-head attention
- Fixed numerical values so every run is reproducible.


In [16]:

# Fixed toy inputs (3 tokens, 4-dim embeddings)
X = torch.tensor([
    [0.5, 0.2, 0.1, -0.1],
    [0.0, 0.3, -0.2, 0.2],
    [0.4, -0.1, 0.0, 0.3]
], dtype=torch.float32)

# Fixed projection matrices
W_Q = torch.tensor([
    [0.2, -0.1, 0.0, 0.3],
    [0.1,  0.0, 0.2, -0.2],
    [-0.1, 0.3, 0.1, 0.0],
    [0.0,  0.2, -0.2, 0.1]
], dtype=torch.float32)

W_K = torch.tensor([
    [0.1,  0.2, 0.0, -0.1],
    [0.0,  0.1, 0.3,  0.0],
    [0.2, -0.2, 0.1,  0.1],
    [-0.1, 0.0, 0.2,  0.2]
], dtype=torch.float32)

W_V = torch.tensor([
    [0.3,  0.1, 0.0, -0.2],
    [0.0,  0.2, 0.1,  0.0],
    [0.1, -0.1, 0.2,  0.1],
    [0.0,  0.1, -0.2, 0.3]
], dtype=torch.float32)

d_k = 4
scale = 1.0 / math.sqrt(d_k)

# Simple target for the loss: same shape as A
T = torch.tensor([
    [0.10, 0.00, 0.05, -0.05],
    [0.00, 0.10, -0.05, 0.05],
    [0.05, -0.05, 0.10, 0.00]
], dtype=torch.float32)

print(f"X = \n{X}\nW_Q.shape = {W_Q.shape}\nW_K.shape = {W_K.shape}\nW_V.shape = {W_V.shape}\nT.shape = {T.shape}")


X = 
tensor([[ 5.000e-01,  2.000e-01,  1.000e-01, -1.000e-01],
        [ 0.000e+00,  3.000e-01, -2.000e-01,  2.000e-01],
        [ 4.000e-01, -1.000e-01,  0.000e+00,  3.000e-01]])
W_Q.shape = torch.Size([4, 4])
W_K.shape = torch.Size([4, 4])
W_V.shape = torch.Size([4, 4])
T.shape = torch.Size([3, 4])



## 2. Forward Pass

We implement scaled dot-product self-attention:


$Q = X W_Q, \quad$
$K = X W_K, \quad$
$V = X W_V$


$S = \frac{QK^\top}{\sqrt{d_k}}, \quad$
$P = \text{softmax}(S) \text{ (row-wise)}, \quad$
$A = P V$

Loss:
$\mathcal{L} = \tfrac12 \|A - T\|^2.$


In [17]:
# Clone parameters with requires_grad for autograd
X_ag  = X.clone().detach().requires_grad_(True)
WQ_ag = W_Q.clone().detach().requires_grad_(True)
WK_ag = W_K.clone().detach().requires_grad_(True)
WV_ag = W_V.clone().detach().requires_grad_(True)

def attention_forward(X_, WQ_, WK_, WV_):
    Q = X_ @ WQ_
    K = X_ @ WK_
    V = X_ @ WV_
    S = (Q @ K.transpose(0, 1)) * scale
    P = torch.softmax(S, dim=-1)
    A = P @ V
    return Q, K, V, S, P, A

Q_ag, K_ag, V_ag, S_ag, P_ag, A_ag = attention_forward(X_ag, WQ_ag, WK_ag, WV_ag)
L_ag = 0.5 * torch.sum((A_ag - T)**2)

print("Attention output A:")
print(A_ag)
print(f"\nLoss L: {L_ag:0.3e}")

Attention output A:
tensor([[ 8.670e-02,  7.332e-02, -1.999e-02, -2.335e-02],
        [ 8.680e-02,  7.331e-02, -1.990e-02, -2.347e-02],
        [ 8.675e-02,  7.329e-02, -2.007e-02, -2.330e-02]],
       grad_fn=<MmBackward0>)

Loss L: 2.861e-02


## 2. Backward Pass (with Autograd)

In [18]:
# Retain gradients for intermediate (non-leaf) tensors so their .grad fields are populated
# when we call backward(). This makes it easy to inspect Q_ag.grad, K_ag.grad, etc.
for t in (Q_ag, K_ag, V_ag, S_ag, P_ag, A_ag):
    # Only call retain_grad() on tensors that require grad and are not leafs
    if isinstance(t, torch.Tensor) and t.requires_grad and not t.is_leaf:
        t.retain_grad()

L_ag.backward()

def safe_clone_grad(tensor):
    if tensor is None:
        return None
    g = tensor.grad
    if g is None:
        return None
    return g.detach().clone()

grads_auto = {
    "X": safe_clone_grad(X_ag),
    "W_Q": safe_clone_grad(WQ_ag),
    "W_K": safe_clone_grad(WK_ag),
    "W_V": safe_clone_grad(WV_ag),
    "Q": safe_clone_grad(Q_ag),
    "K": safe_clone_grad(K_ag),
    "V": safe_clone_grad(V_ag),
    "S": safe_clone_grad(S_ag),
    "P": safe_clone_grad(P_ag),
    "A": safe_clone_grad(A_ag),
}

print("\nAutograd gradient dL/dW_Q:")
print(grads_auto["W_Q"])
print("\nAutograd gradient dL/dW_K:")
print(grads_auto["W_K"])
print("\nAutograd gradient dL/dW_V:")
print(grads_auto["W_V"])


Autograd gradient dL/dW_Q:
tensor([[-1.272e-04, -3.362e-05,  3.323e-05,  8.967e-05],
        [ 7.927e-05,  1.711e-05, -3.356e-05, -4.563e-05],
        [-9.281e-05, -2.068e-05,  3.714e-05,  5.514e-05],
        [ 6.987e-05,  1.308e-05, -3.626e-05, -3.487e-05]])

Autograd gradient dL/dW_K:
tensor([[-1.752e-05, -4.336e-06, -2.968e-05, -1.466e-04],
        [-2.176e-05,  1.324e-06,  1.287e-05, -1.607e-05],
        [-1.367e-05, -2.544e-06, -1.694e-05, -9.365e-05],
        [ 3.736e-05,  2.116e-06,  1.044e-05,  1.362e-04]])

Autograd gradient dL/dW_V:
tensor([[ 3.312e-02,  5.101e-02, -4.801e-02, -2.107e-02],
        [ 1.468e-02,  2.260e-02, -2.127e-02, -9.339e-03],
        [-3.652e-03, -5.654e-03,  5.323e-03,  2.319e-03],
        [ 1.469e-02,  2.269e-02, -2.137e-02, -9.335e-03]])


## 3. Manual Backpropagation

We recompute the forward pass without autograd and apply the analytical formulas.

Start with:
$\frac{\partial \mathcal{L}}{\partial A} = A - T$

Then:

1. Through $A = P V$:
$\frac{\partial \mathcal{L}}{\partial V} = P^\top \frac{\partial \mathcal{L}}{\partial A}, \quad$
$\frac{\partial \mathcal{L}}{\partial P} = \frac{\partial \mathcal{L}}{\partial A} V^\top$

2. Softmax (row-wise for each query $i$):
$
\frac{\partial \mathcal{L}}{\partial s_i} = J_{\text{softmax}}(p_i)^{\top}
\frac{\partial \mathcal{L}}{\partial p_i},
\quad
J_{\text{softmax}}(p_i) = \mathrm{diag}(p_i) - p_i p_i^\top
$

3. Through $S = QK^\top / \sqrt{d_k}$:
$
\frac{\partial \mathcal{L}}{\partial Q}
= \frac{1}{\sqrt{d_k}} \frac{\partial \mathcal{L}}{\partial S} K,
\quad
\frac{\partial \mathcal{L}}{\partial K}
= \frac{1}{\sqrt{d_k}}
\left(\frac{\partial \mathcal{L}}{\partial S}\right)^{\top} Q
$

4. Through projections:
   
$
\frac{\partial \mathcal{L}}{\partial W_Q} = X^\top \frac{\partial \mathcal{L}}{\partial Q},\;
\frac{\partial \mathcal{L}}{\partial W_K} = X^\top \frac{\partial \mathcal{L}}{\partial K},\;
\frac{\partial \mathcal{L}}{\partial W_V} = X^\top \frac{\partial \mathcal{L}}{\partial V}
$

$ \frac{\partial \mathcal{L}}{\partial X} = \frac{\partial \mathcal{L}}{\partial Q} W_Q^\top + \frac{\partial \mathcal{L}}{\partial K} W_K^\top + \frac{\partial \mathcal{L}}{\partial V} W_V^\top $

In [ ]:

# Forward without autograd
with torch.no_grad():
    Q = X @ W_Q
    K = X @ W_K
    V = X @ W_V
    S = (Q @ K.t()) * scale
    P = torch.softmax(S, dim=-1)
    A = P @ V
    L = 0.5 * torch.sum((A - T)**2)

# 1) dL/dA
dA = (A - T)

# 2) Through A = P V
dV = P.t() @ dA
dP = dA @ V.t()

# 3) Softmax Jacobian row-wise
def softmax_grad_row(p_row, dp_row):
    # p_row, dp_row: 1D tensors of length n
    p = p_row.view(-1, 1)
    J = torch.diagflat(p) - p @ p.t()
    return (J.t() @ dp_row.view(-1, 1)).view(-1)

n = P.shape[0]
dS = torch.zeros_like(S)
for i in range(n):
    dS[i] = softmax_grad_row(P[i], dP[i])

# 4) Through S = (QK^T)/sqrt(d_k)
dQ = (dS @ K) * scale
dK = (dS.t() @ Q) * scale

# 5) Through projections
dW_Q = X.t() @ dQ
dW_K = X.t() @ dK
dW_V = X.t() @ dV

dX = dQ @ W_Q.t() + dK @ W_K.t() + dV @ W_V.t()

grads_manual = {
    "X": dX,
    "W_Q": dW_Q,
    "W_K": dW_K,
    "W_V": dW_V,
    "Q": dQ,
    "K": dK,
    "V": dV,
    "S": dS,
    "P": dP,
    "A": dA,
}

print(f"Loss from manual forward: {L:.{PRECISION}e}")


Loss from manual forward: 2.8612e-02



## 4. Compare Manual vs Autograd Gradients


In [ ]:

def max_abs_diff(a, b):
    return float(torch.max(torch.abs(a - b)))

for name in ["W_Q", "W_K", "W_V", "X"]:
    diff = max_abs_diff(grads_manual[name], grads_auto[name])
    print(f"Max |manual - autograd| for {name}: {diff:{PRECISION}.e}")

print("\nAutograd dL/dW_Q:\n", grads_auto["W_Q"])
print("\nManual   dL/dW_Q:\n", grads_manual["W_Q"])


Max |manual - autograd| for W_Q: 2.546585e-11
Max |manual - autograd| for W_K: 2.910383e-11
Max |manual - autograd| for W_V: 0.000000e+00
Max |manual - autograd| for X: 1.862645e-09

Autograd dL/dW_Q:
 tensor([[-1.272e-04, -3.362e-05,  3.323e-05,  8.967e-05],
        [ 7.927e-05,  1.711e-05, -3.356e-05, -4.563e-05],
        [-9.281e-05, -2.068e-05,  3.714e-05,  5.514e-05],
        [ 6.987e-05,  1.308e-05, -3.626e-05, -3.487e-05]])

Manual   dL/dW_Q:
 tensor([[-1.272e-04, -3.362e-05,  3.323e-05,  8.967e-05],
        [ 7.927e-05,  1.711e-05, -3.356e-05, -4.563e-05],
        [-9.281e-05, -2.068e-05,  3.714e-05,  5.514e-05],
        [ 6.987e-05,  1.308e-05, -3.626e-05, -3.487e-05]])



## 5. Finite-Difference Check

Pick one element of $W_Q$ and approximate its gradient numerically.


In [ ]:
def compute_loss_with_WQ(WQ_new):
    Q = X @ WQ_new
    K = X @ W_K
    V = X @ W_V
    S = (Q @ K.t()) * scale
    P = torch.softmax(S, dim=-1)
    A = P @ V
    return 0.5 * torch.sum((A - T)**2)

i, j = 0, 0  # test this entry
eps = 1e-4

with torch.no_grad():
    WQ_plus = W_Q.clone();  WQ_plus[i, j] += eps
    WQ_minus = W_Q.clone(); WQ_minus[i, j] -= eps
    L_plus = compute_loss_with_WQ(WQ_plus)
    L_minus = compute_loss_with_WQ(WQ_minus)
    num_grad = (L_plus - L_minus) / (2 * eps)

print(f"Finite-diff grad for W_Q[{i},{j}]: {float(num_grad):.2e}")
print(f"Autograd grad: {float(grads_auto['W_Q'][i,j]):.2e}")
print(f"Manual grad: {float(grads_manual['W_Q'][i,j]):.2e}")


Finite-diff grad for W_Q[0,0]: -1.397e-04
Autograd grad: -1.272e-04
Manual grad: -1.272e-04



## 6. What to Notice

- Manual gradients match autograd (up to numerical precision).
- Each query position's loss sends signal to **all** keys/values it attends to.
- The softmax Jacobian step is where this coupling happens.
- The $1/\sqrt{d_k}$ factor keeps scores and gradients in a stable range.

## 7. Do the following
- Change the target $T$ or matrices and re-run cells.
- Predict qualitatively how attention weights and gradients will shift.
